In [ ]:
# Import the Image class from the PIL (Pillow) library
from PIL import Image

# Import the requests library to make HTTP requests
import requests

# Define the URL of the image you want to download
url = "http://images.cocodataset.org/val2017/000000039769.jpg"

# Use the requests library to download the image and open it with PIL
image = Image.open(requests.get(url, stream=True).raw)

# Display the image
image.show()

In [ ]:
# Import the necessary classes from the transformers library
from transformers import GroundingDinoProcessor, GroundingDinoForObjectDetection

# Initialize the processor using a pre-trained model checkpoint
processor = GroundingDinoProcessor.from_pretrained("IDEA-Research/grounding-dino-base")

# Initialize the object detection model using the same pre-trained model checkpoint
model = GroundingDinoForObjectDetection.from_pretrained("IDEA-Research/grounding-dino-base")

In [ ]:
# Define a text caption
text = "a cat"

# Define a function to preprocess the caption text
def preprocess_caption(caption: str) -> str:
    # Convert the caption to lowercase and remove leading/trailing whitespace
    result = caption.lower().strip()
    # Check if the caption ends with a period
    if result.endswith("."):
        return result
    # If not, append a period to the caption
    return result + "."

# Preprocess the caption text
processed_text = preprocess_caption(text)

# Use the processor to prepare the inputs for the model
inputs = processor(images=image, text=processed_text, return_tensors="pt")

In [ ]:
# Import the torch library
import torch

# Perform inference without computing gradients (to save memory and computation)
with torch.no_grad():
    # Get the outputs from the model by passing the processed inputs
    outputs = model(**inputs)

In [ ]:
# Import the matplotlib library for visualization
import matplotlib.pyplot as plt

# Define colors for bounding box visualization
COLORS = [
    [0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
    [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]
]

# Define a function to plot the results
def plot_results(pil_img, scores, labels, boxes):
    # Set the size of the plot
    plt.figure(figsize=(16, 10))
    
    # Display the image
    plt.imshow(pil_img)
    
    # Get the current axis
    ax = plt.gca()
    
    # Repeat the colors to ensure there are enough for all boxes
    colors = COLORS * 100
    
    # Iterate over the scores, labels, boxes, and colors
    for score, label, (xmin, ymin, xmax, ymax), c in zip(scores, labels, boxes, colors):
        # Add a rectangle for each bounding box
        ax.add_patch(plt.Rectangle(
            (xmin, ymin), xmax - xmin, ymax - ymin,
            fill=False, color=c, linewidth=3
        ))
        
        # Create a label with the text and score
        label = f'{label}: {score:0.2f}'
        
        # Add the label to the plot
        ax.text(xmin, ymin, label, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    
    # Remove the axis
    plt.axis('off')
    
    # Show the plot
    plt.show()

In [ ]:
# Get the dimensions of the image
width, height = image.size

# Post-process the model outputs to get the final object detection results
postprocessed_outputs = processor.image_processor.post_process_object_detection(
    outputs,                     # The raw outputs from the model
    target_sizes=[(height, width)],  # The target size of the image as a tuple (height, width)
    threshold=0.3                # Confidence threshold for filtering detections
)

# Extract the results for the first image (since we're working with a single image)
results = postprocessed_outputs[0]

In [ ]:
# Call the plot_results function to visualize the detection results
plot_results(
    image,                      # The original image
    results['scores'].tolist(), # List of detection scores
    results['labels'].tolist(), # List of detected labels
    results['boxes'].tolist()   # List of bounding boxes
)

In [ ]:
# Import the pipeline function from the transformers library
from transformers import pipeline

# Initialize a zero-shot object detection pipeline using the specified model and device
pipe = pipeline(task="zero-shot-object-detection", model="IDEA-Research/grounding-dino-base", device='cuda:0')

# Define the URL of the image to be processed
image_url = 'http://images.cocodataset.org/val2017/000000039769.jpg'

# Define the candidate labels (preprocess the text)
candidate_labels = [preprocess_caption(text)]

# Use the pipeline to perform zero-shot object detection on the image
results = pipe(image_url, candidate_labels=candidate_labels, threshold=0.3)

# Print the results
print(results)


In [ ]:
# Initialize empty lists to store scores, labels, and boxes
scores, labels, boxes = [], [], []

# Iterate over the detection results
for result in results:
    # Append the score to the scores list
    scores.append(result["score"])
    
    # Append the label to the labels list
    labels.append(result["label"])
    
    # Append the bounding box coordinates as a tuple to the boxes list
    boxes.append(tuple(result["box"].values()))

# Call the plot_results function to visualize the detection results
plot_results(image, scores, labels, boxes)
